# The goal of this notebook is to detect and segment holes wihtin coacervates to quantifiy the hole are vs droplet area

## 1. Import all libraries

In [1]:
import os
import czifile
import skimage

import matplotlib.pyplot as plt
import pyclesperanto_prototype as cle
import matplotlib
import numpy as np
import pandas as pd
from skimage.measure import regionprops
from skimage.morphology import remove_small_objects
from skimage.morphology import convex_hull_image
from skimage.io import imread, imsave
from skimage.transform import hough_circle, hough_circle_peaks
from skimage import filters
from skimage.draw import circle_perimeter
from skimage import data, color
from skimage import morphology
from skimage.filters import gaussian, median, threshold_otsu, threshold_multiotsu, prewitt
from skimage.morphology import skeletonize, binary_erosion
from skimage import measure
from skimage.segmentation import watershed

from scipy import ndimage
from scipy.stats import pearsonr
from scipy.stats import spearmanr

import napari
from napari_simpleitk_image_processing import touching_objects_labeling
from napari_simpleitk_image_processing import label_statistics # to check if it is needed
from ismember import ismember


import seaborn as sns


/Users/wverstra/miniconda3/envs/seqenv/lib/python3.12/site-packages/napari_tools_menu/__init__.py:10: UserWarning: Importing QT failed; now introducing dummy definitions of QMenu class and register_function decorator.
  warnings.warn("Importing QT failed; now introducing dummy definitions of QMenu class and register_function decorator.")
/Users/wverstra/miniconda3/envs/seqenv/lib/python3.12/site-packages/napari_tools_menu/__init__.py:221: UserWarning: Error in monkey patching napari. Please let @haesleinleinhuepf know at
https://github.com/haesleinhuepf/napari-tools-menu/issues
  warnings.warn("Error in monkey patching napari. Please let @haesleinleinhuepf know at\nhttps://github.com/haesleinhuepf/napari-tools-menu/issues")
/Users/wverstra/miniconda3/envs/seqenv/lib/python3.12/site-packages/napari_skimage_regionprops/_table.py:7: UserWarning: No Qt bindings could be found
  warnings.warn(str(e))


#### Use cafeine to prevent laptop from going to sleep

In [12]:
pip install caffeine

Note: you may need to restart the kernel to use updated packages.


In [13]:
import caffeine
caffeine.on(display=True)

## 2. we define all the functions

In [14]:
def detection(binary):
    #Label the images to exclude structures on the edges

    labeled_image = cle.connected_components_labeling_box(binary)
    #This first detects connected objects after the thresholding

    labeled_edges_image = cle.exclude_labels_on_edges(labeled_image)   
    #Now we can detect entire objects that are touching the edges,
    #note that we have to do this after turning pixels into objects
    
    return labeled_edges_image

In [15]:
def calculate_area_including_empty_regions(labelled_image):
    """
    Calculates the area of coacervates, including any hollow or empty regions within them,
    and returns the areas as a numpy array.
    
    Parameters:
        labelled_image: Labeled image where each connected object has a unique label.
    
    Returns:
        areas: NumPy array containing the area for each labeled coacervate, including empty regions.
    """
    # Get properties of the labeled regions (coacervates)
    props = regionprops(labelled_image)
    
    # List to store areas of each coacervate
    areas = []
    
    for prop in props:
        # Get the convex hull of the coacervate (this includes any hollow regions within)
        convex_hull = convex_hull_image(labelled_image == prop.label)
        
        # Calculate the area of the convex hull (this includes the hollow regions)
        area_including_holes = np.sum(convex_hull)
        
        # Append the area to the list
        areas.append(area_including_holes)
    
    # Convert list to NumPy array
    return np.array(areas)


In [36]:

from skimage.measure import regionprops


def plot_labeled_coacervates(labelled_image, RNA_image,save_path="labeled_coacervates.png"):
    """
    Plots the detected coacervates in grayscale with their corresponding labels next to each object,
    and saves the output as a PNG file.
    
    Parameters:
        labelled_image: Labeled image where each connected object has a unique label.
        save_path: Path to save the PNG image (default is 'labeled_coacervates.png').
    """
    # Normalize the labeled image to make the coacervates brighter
    bright_image = (labelled_image / np.max(labelled_image))*255  # Normalize the labels between 0 and 1
    RGB_image = np.zeros((np.shape(bright_image)[0], np.shape(bright_image)[1], 3))
    print(np.shape(RGB_image))
    
    RGB_image[:,:,2] =  np.array(bright_image)
    RGB_image[:,:,1] = RNA_image
    
    
    
    
    
    # Create a figure for plotting
    plt.figure(figsize=(10, 10))
    
    # Plot the grayscale image of the labeled coacervates
    #plt.imshow(bright_image, cmap ="viridis")
    plt.imshow(RGB_image)
    #plt.show()



    # Get properties of the labeled regions
    props = regionprops(labelled_image)
    
    # Loop over each region and add its label next to the coacervate's centroid
    for prop in props:
        y, x = prop.centroid  # Get the centroid of the coacervate
        label = prop.label    # Get the label of the coacervate
        plt.text(x, y, str(label), color="red", fontsize=12, fontweight='bold')
    
    # Add title and remove axes
    plt.title("Detected Coacervates with Labels", fontsize=16)
    plt.axis("off")
    plt.draw()
    
    # Save the figure as a PNG file
    plt.savefig(save_path, bbox_inches='tight', dpi=300)
    
    # Display the plot
    plt.show()



## 3. Now we can analyze the compilation of droplets

In [ ]:
folder = "" # Link to where the tif or czi files are stored
size_x = 639 #image size in micro meters (only different for RNA nanostars with a TL)
size_y = 639
output_file = '/Users/XXXX/1.csv' # 
output_folder = "/Users/XXXX/Segmentation/1/" # Create a separate folder to save the overlay of the detected ROIs and the fluorescent dropelts images
counter = 0
for time in os.listdir(folder):
    t = time[:2]
    time_folder = folder + time + "/"
    for condition in os.listdir(time_folder):
        c = condition
        condition_folder = time_folder + str(condition) + "/"
        for replicate in os.listdir(condition_folder):
            R = replicate
            replicate_folder = condition_folder + str(R) + "/"
            for img in os.listdir(replicate_folder):
                img_path = replicate_folder + str(img)
                print(img_path)

                if ".tif" in (img):
                    counter += 1

                    i = imread(img_path)
                    # I need to reshape the image because some image have different resolutions #however the total size in um is always the same
                    print(np.shape(i))
                    #however the total size in um is always the same, so I can easily calculate the pixel size
                    px_size = size_x/np.shape(i)[1]
                    #BF_image = i[-1,:,:]
                    RNA_image = i[-2,:,:]
                    # We will use the RNA image to detect droplets and holes in them
                    # Apply gaussian blurring
                    RNA_image = RNA_image/np.max(RNA_image)
                    smoothed = gaussian(RNA_image, sigma = 5, preserve_range = True)
                    threshold = threshold_multiotsu(smoothed, classes = 2) 
                    binary = RNA_image > 0.8 * threshold[0]
    
                    # Now to remove small objects
                    min_object_size = int(((np.shape(i)[2])/1024)*90)
                    cleaned_binary = remove_small_objects(binary, min_size=min_object_size)
    
        
                    # Now from the binary image we get the labelled image
                    labelled_coacervates = detection(cleaned_binary)
                    # Now we will save the image so I can judge how well the segmenration was done
    
                    #plot_labeled_coacervates(labelled_coacervates)                
                    #plt.savefig(output_folder + str(img) + "_overlay"+ ".png")
                    plot_labeled_coacervates(labelled_coacervates, RNA_image,output_folder + str(img) + "_overlay"+ ".png")
                    # Using this labelled image we can easily measure different properties for each detected coacervate
                    properties = ['label','area','perimeter', 'axis_major_length', 'axis_minor_length', 'intensity_mean', 'area_filled']
                    # Note that the intensity of the detected GUVs is measured on the DNA image!!
                    df=pd.DataFrame(
                    measure.regionprops_table(np.array(labelled_coacervates),intensity_image= RNA_image, 
                    properties=properties), columns = properties)
                    #df['full area'] = calculate_area_including_empty_regions(labelled_coacervates)
                    df['hollow area'] = df['area_filled'] - df['area']
                    # Calculate the different area ratios
                    df['area ratios'] = df['hollow area']/df['area_filled']
                    df['pixel size'] = px_size
                    df['time'] = t
                    df['condition'] = condition
                    df['image_ID'] = img
                    df['replicate'] = R
                    if counter>1:
                        df.to_csv(output_file, mode='a',index=True, header=False)
                    else:
                        df.to_csv(output_file, mode='w',index=True,header = True)
                
                elif ".czi" in(img):   
                    
                    counter += 1

                    i = czifile.imread(img_path)

                    # I need to reshape the image because some image have different resolutions #however the total size in um is always the same
                    print(np.shape(i))
                    i = i.reshape((np.shape(i)[2],np.shape(i)[4],np.shape(i)[5])) 
                    #however the total size in um is always the same, so I can easily calculate the pixel size
                    px_size = size_x/np.shape(i)[2]
                    #BF_image = i[-1,:,:]
                    RNA_image = i[-2,:,:]
                    # We will use the RNA image to detect droplets and holes in them
                    # Apply gaussian blurring
                    RNA_image = RNA_image/np.max(RNA_image)
                    smoothed = gaussian(RNA_image, sigma = 5, preserve_range = True)
                    threshold = threshold_multiotsu(smoothed, classes = 2) 
                    binary = RNA_image > 0.8 * threshold[0]
    
                    # Now to remove small objects
                    min_object_size = int(((np.shape(i)[2])/1024)*90)
                    cleaned_binary = remove_small_objects(binary, min_size=min_object_size)
    
        
                    # Now from the binary image we get the labelled image
                    labelled_coacervates = detection(cleaned_binary)
                    # Now we will save the image so I can judge how well the segmenration was done
    
                    #plot_labeled_coacervates(labelled_coacervates)                
                    #plt.savefig(output_folder + str(img) + "_overlay"+ ".png")
                    plot_labeled_coacervates(labelled_coacervates, RNA_image,output_folder + str(img) + "_overlay"+ ".png")
                    # Using this labelled image we can easily measure different properties for each detected coacervate
                    properties = ['label','area','perimeter', 'axis_major_length', 'axis_minor_length', 'intensity_mean', 'area_filled']
                    # Note that the intensity of the detected GUVs is measured on the DNA image!!
                    df=pd.DataFrame(
                    measure.regionprops_table(np.array(labelled_coacervates),intensity_image= RNA_image, 
                    properties=properties), columns = properties)
                    #df['full area'] = calculate_area_including_empty_regions(labelled_coacervates)
                    df['hollow area'] = df['area_filled'] - df['area']
                    # Calculate the different area ratios
                    df['area ratios'] = df['hollow area']/df['area_filled']
                    df['pixel size'] = px_size
                    df['time'] = t
                    df['condition'] = condition
                    df['image_ID'] = img
                    df['replicate'] = R
                    if counter>1:
                        df.to_csv(output_file, mode='a',index=True, header=False)
                    else:
                        df.to_csv(output_file, mode='w',index=True,header = True)


## 4. Now for the timelapses we need to do things slightly differently

In [ ]:
# Name of the folder where the timelapses are stored
# Make sure your timelapse is saved as a .tif file and only contains one "tile" region
size_x = 639 #image size in micro meters
size_y = 639
output_file = '/Users/XXXX/1.csv' # 
folder = '/Users/wverstra/Desktop/RNA droplets stuff/second timelapse/Will_data/'
output_file = "/Users/XXXX/timelapse.csv"
#output_folder = "/Users/wverstra/Documents/Segmentation/Timelapse Wa/" # You can also save the overlays of the timelapse if you want to 

size_x = 639
size_y = 639
counter = 0

for time in os.listdir(folder):
    time_folder = folder + time + "/"
    print(time_folder)
    for field in os.listdir(time_folder):
        f = field
        condition_folder = time_folder + str(field) + "/"
        for img in os.listdir(condition_folder):
            img_path = condition_folder + str(img)
            print(img_path)
            print(field)
            if ".tif" in(img):   
                i = imread(img_path)
                # I need to reshape the image because some image have different resolutions 
                print(np.shape(i))
                px_size = size_x/np.shape(i)[2]

                for t in range(len(i)):
                    print(t)

                    RNA_image = i[t,:,:]
                    #however the total size in um is always the same, so we can easily calculate the pixel size
                    # We will use the RNA image to detect droplets and holes in them
                    # Apply gaussian blurring
                    RNA_image = RNA_image/np.max(RNA_image)
                    smoothed = gaussian(RNA_image, sigma = 1, preserve_range = True)
                    threshold = threshold_multiotsu(smoothed, classes = 2) 
                    binary = RNA_image > 0.8 * threshold[0] 
                    # Might need to chane this treshold depending on your images
    
                    # Now to remove small objects
                    min_object_size = int(((np.shape(i)[2])/1024)*90)
                    #Again. might need to change the exact thresholding value based on your objects of interest
                    cleaned_binary = remove_small_objects(binary, min_size=min_object_size)
    
        
                    # Now from the binary image we get the labelled image
                    labelled_coacervates = detection(cleaned_binary)
                    
                    # Now we will save the image so you can judge how well the segmenration was done 
                    
                    
                    #(uncoment the next three lines if you want to save the overlay)
                    #plot_labeled_coacervates(labelled_coacervates)                
                    #plt.savefig(output_folder + str(img) + "_overlay"+ ".png")
                    #plot_labeled_coacervates(labelled_coacervates, RNA_image,output_folder + str(img) + "_overlay_"+ str(counter) + "_.png")
                    
                    counter +=1

                    # Using this labelled image we can easily measure different properties for each detected coacervate
                    properties = ['label','area','perimeter', 'axis_major_length', 'axis_minor_length', 'intensity_mean', 'area_filled']
                    # Note that the intensity of the detected GUVs is measured on the DNA image!!
                    df=pd.DataFrame(
                    measure.regionprops_table(np.array(labelled_coacervates),intensity_image= RNA_image, 
                    properties=properties), columns = properties)
                    #df['full area'] = calculate_area_including_empty_regions(labelled_coacervates)
                    df['hollow area'] = df['area_filled'] - df['area']
                    # Calculate the different area ratios
                    df['area ratios'] = df['hollow area']/df['area_filled']
                    df['pixel size'] = px_size
                    df['time'] = time
                    df['frame'] = t
                    df['condition'] = field
                    df['image_ID'] = img
                    df['field'] = f
                    if counter>1:
                        df.to_csv(output_file, mode='a',index=True, header=False)
                        print("There should already be a CSV")
                    else:
                        df.to_csv(output_file, mode='w',index=True,header = True)
                        print("No csv yet")
